In [ ]:
%pip install qdrant_client

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import torch
import numpy as np
from urllib.parse import urlparse


In [ ]:
qdranturl = 'qdrant://localhost:6333#test_qdrant'


In [ ]:
o = urlparse(qdranturl, allow_fragments=True)

In [ ]:
print(
    o.scheme,
    o.hostname,
    o.port,
    o.path,
    o.fragment,
)

In [ ]:
client = QdrantClient(host=o.hostname, port=o.port, grpc_port=(o.port+1), prefer_grpc=True)


In [ ]:
if client.collection_exists(collection_name=o.fragment):
    print('collection exists')
    collection_info = client.get_collection(collection_name=o.fragment)
    print(list(collection_info))
    print(collection_info.status)
    print(collection_info.vectors_count)
else:
    print('collection does not exist')
    client.create_collection(
        collection_name=o.fragment,
        vectors_config=models.VectorParams(size=5, distance=models.Distance.COSINE),
    )

In [ ]:
a = torch.rand(int(1e4),5)
print(a.size())


In [ ]:
## add data
operation_info = client.upsert(
    collection_name=o.fragment,
    wait=True,
    points=[
        models.PointStruct(id=1, vector=a[0]),
        models.PointStruct(id=2, vector=a[1]),
    ]
)
print(operation_info)



In [ ]:
operation_info = client.upsert(
    collection_name=o.fragment,
    wait=True,
    points=models.Batch(
        ids=list(range(int(1e4))),
        vectors=a
    )
)
print(operation_info)

In [ ]:
result = client.retrieve(
    collection_name=o.fragment,
    ids=[12],
    with_vectors=True,
    with_payload=False
)

In [ ]:
print(len(result))
print(result[0].id)
print(torch.tensor(result[0].vector, dtype=torch.float32))

In [ ]:
collection_info = client.get_collection(collection_name=o.fragment)
print(collection_info)